# Error analysis for the type of examples that Punctuation and Digit get wrong compared to the baseline

In [2]:
%load_ext autoreload

In [41]:
%autoreload
import os
from sys import path
import re
import pandas as pd
path.append('../..')
import numpy as np
from scipy.stats import ttest_rel
from relation_extraction.data.summarize import *
from relation_extraction.data.converters.converter_ddi import relation_dict
output_path = '/scratch/geeticka/relation-extraction-result/ddi-analyze/'
def res(path): return os.path.join(output_path, path)
original_sentences_path = os.path.join('/scratch/geeticka/relation-extraction-result/ddi-analyze/test_original.txt')
punct_digit_sentences_path = os.path.join('/crimea/geeticka/data/relation_extraction/ddi/pre-processed/punct_digit/test_punct_digit.txt')
relation_full_form_dictionary = {'advise': 'Advise', 'effect': 'Effect', 'mechanism': 'Mechanism',
                                 'int': 'Interaction', 'none': 'None'}
relation_as_short_list = ['advise', 'effect', 'mechanism', 'int', 'none']

In [42]:
relation_dict

{0: 'advise', 1: 'effect', 2: 'mechanism', 3: 'int', 4: 'none'}

In [43]:
# confusion_matrix_portion = '<<< 8-WAY EVALUATION >>>'

In [44]:
# summary_cm_baseline, summary_cm_conceptblind = print_full_summary('i2b2',
#     'result-baseline-i2b2.txt', 'result-concblind-i2b2.txt', 
#     'Baseline','Concept-Blind', res, relation_full_form_dictionary, relation_as_short_list, confusion_matrix_portion, 2)

In [45]:
# summary_cm_baseline

In [46]:
# summary_cm_conceptblind

In [47]:
def read_answers_line(line):
    linenum, relation = line.strip().split()
    return linenum, relation

In [48]:
def asstring(list_of_strings):
    return " ".join(list_of_strings)

In [49]:
def read_sentence_and_entities(line):
    line = line.strip().split()
    sentence = line[5:]
    relation = relation_dict[int(line[0])]
    entity1_idx = (int(line[1]), int(line[2]))
    entity2_idx = (int(line[3]), int(line[4]))
    entity1 = sentence[entity1_idx[0] : entity1_idx[1] + 1]
    entity2 = sentence[entity2_idx[0] : entity2_idx[1] + 1]
    
    return relation, asstring(entity1), asstring(entity2), asstring(sentence)

In [50]:
needed_linenum_and_relation = {}

In [51]:
with open(res("answers_for_dev-punct-digit.txt")) as textfile1, open(res('answers_for_dev-baseline.txt')) as textfile2, \
open(res('answers_for_dev-baseline_gold.txt')) as textfile3: 
    for x, y, z in zip(textfile1, textfile2, textfile3):
        linenum, punct_digit_relation = read_answers_line(x)
        _, baseline_relation = read_answers_line(y)
        _, gold_relation = read_answers_line(z)
        if baseline_relation == gold_relation and punct_digit_relation != gold_relation: # punct digit making mistakes
            needed_linenum_and_relation[int(linenum) - 1] = (baseline_relation, punct_digit_relation)

In [52]:
len(list(needed_linenum_and_relation.keys()))

150

Remember that baseline is correct here so the first relation listed is the correct one, unlike in the i2b2 notebook where the order is reversed because baseline is the incorrect one there

In [57]:
print('We print the baseline first and then the punct digit version. Gold relation corresponds to baseline\n\n')
curr_linenum = 0
with open(original_sentences_path) as original_sentences, open(punct_digit_sentences_path) as punct_digit_sentences:
    for x, y in zip(original_sentences, punct_digit_sentences):
        needed_linenums = list(needed_linenum_and_relation.keys())
        if curr_linenum in needed_linenums:
            _, e1_b, e2_b, s_b = read_sentence_and_entities(x.strip())
            _, e1_c, e2_c, s_c = read_sentence_and_entities(y.strip())
            r_b, r_c = needed_linenum_and_relation[curr_linenum]
            print('Predicted Relation: \t {0}, {1} \nEntities: \t {2}, {3} \nSentences: \n\t{4} \n\t {5}'.format(
                    r_b, r_c, e1_b, e2_b, s_b, s_c))
            print('\n')
        curr_linenum += 1

We print the baseline first and then the punct digit version. Gold relation corresponds to baseline


Predicted Relation: 	 mechanism, none 
Entities: 	 Terbinafine, rifampin 
Sentences: 
	Terbinafine clearance is increased 100 % by rifampin , a CyP450 enzyme inducer , and decreased 33 % by cimetidine , a CyP450 enzyme inhibitor . 
	 Terbinafine clearance is increased NUMBER by rifampin a CyP450 enzyme inducer and decreased NUMBER by cimetidine a CyP450 enzyme inhibitor


Predicted Relation: 	 mechanism, none 
Entities: 	 Terbinafine, cimetidine 
Sentences: 
	Terbinafine clearance is increased 100 % by rifampin , a CyP450 enzyme inducer , and decreased 33 % by cimetidine , a CyP450 enzyme inhibitor . 
	 Terbinafine clearance is increased NUMBER by rifampin a CyP450 enzyme inducer and decreased NUMBER by cimetidine a CyP450 enzyme inhibitor


Predicted Relation: 	 mechanism, none 
Entities: 	 gemfibrozil, rosiglitazone 
Sentences: 
	An inhibitor of CYP2C8 ( such as gemfibrozil ) may inc